In [91]:
import pandas as pd

private_data = pd.read_excel('private_dataM.xlsx')

In [92]:
anon = private_data.copy()

In [93]:
# remove direct identifiers
anon = anon.drop(columns=['name'])

In [94]:
# generalise quasi-identifiers
'''Age is very specific, so we will transform to age bands, 
and merge further every non-Danish to 'other', merge divorced/widowed, and merge education
 '''

today = pd.Timestamp.today()
anon['dob'] = pd.to_datetime(anon['dob'], format='%d/%m/%Y')
anon['age'] = (today - anon['dob']).dt.days // 365

bins = [18, 29, 39, 49, 59, 69, 120]
labels = ['18-29','30-39','40-49','50-59','60-69', '70+']
anon['age_band'] = pd.cut(anon['age'], bins=bins, labels=labels)
anon = anon.drop(columns=['dob', 'age'])

anon.loc[anon['citizenship'] != 'Denmark', 'citizenship'] = 'Other'
anon.loc[anon['marital_status'].isin(['Divorced', 'Widowed']), 'marital_status'] = 'Divorced/Widowed'

education_map = {
    'Upper secondary education': 'Secondary education',
    'Vocational Education and Training (VET)': 'Secondary education',
    'Vocational bachelors educations': 'Vocational/Medium cycle',
    'Short cycle higher education': 'Vocational/Medium cycle',
    'Bachelors programmes': 'Higher education',
    'Masters programmes': 'Higher education',
    'PhD programmes': 'Higher education',
    'Not stated': 'Other/Not stated',
    'Qualifying educational programmes': 'Other/Not stated'
}

anon['education'] = anon['education'].replace(education_map)

In [ ]:
# supress rare combinations with k-anonymity
# TODO: improve this so we don't remove most of the dataset

test = anon.copy()
qid = [ 'age_band', 'citizenship', 'education', 'sex']
counts = test.groupby(qid).size().reset_index(name='count')
riskydf = test.merge(counts, on=qid, how='left')
risky = riskydf[riskydf['count'].between(1, 3)]
print(len(test), len(risky))

200 71


5     40
4     36
3     33
8     24
1     22
2     16
6     12
10    10
7      7
Name: count, dtype: int64